# Entrega Final

**Therry Jones Bent O'neill, CC 1107433181, Ingeniería Electrica**

**Julián Mateo Mena Urrego, CC 1038821102, Ingeniería Electrica**

**Miguel Angel Rivera Florez, CC 1152467107, Ingeniería Materiales**

#Dataset

https://www.kaggle.com/code/armaanseth6702/predict-co2-emissions-in-rwanda

# Instalacion de modulos y librerias necesarias

In [ ]:
! pip install kaggle

# Librerias Necesarias


Importo las librerias para la modelacion de los datos.

In [ ]:
# Importar librerias
import pandas as pd
import numpy as np
import random
import os
import json
import requests
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.5f}'.format
pd.options.display.max_rows = None

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Lectura de la base de datos

Cargamos las credenciales de Kaggle del compañero y estudiante Julian mena  para facilitar el acceso a los datasets, los cuales son train, test y sample_submission.

In [ ]:
data = {"username":"julianmena42","key":"b7cb043ab70f4bb16bbca4728b2d2e2f"}
with open('kaggle.json', 'w') as file:
    json.dump(data, file, indent=4)

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

#Cargar datos de la dataset

In [ ]:

!kaggle competitions download -c playground-series-s3e20

#Descompresión de archivos

In [ ]:

!unzip playground-series-s3e20.zip

Archive:  playground-series-s3e20.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
# Set seed for reproducability
SEED = 2023
random.seed(SEED)
np.random.seed(SEED)

# 1. Lectura de la base de datos
La base de datos está alojada en el servidor de google drive educativo del estudiante Therry bent, el acceso lo hemos configurado para que la lectura sea pública, y los datasets serán asignados en las variables train, test y sample_submission.

#Asignacion de variables

In [ ]:

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
samplesubmission = pd.read_csv('sample_submission.csv')

In [ ]:
train.head()

In [ ]:
samplesubmission.head()

In [ ]:
test.head()

In [ ]:
train.shape, test.shape, samplesubmission.shape

In [ ]:
# 1. Crear una ubicación única desde latitud y longitud
train['location'] = train['latitude'].round(2).astype(str) + '_' + train['longitude'].round(2).astype(str)

# 2. Filtrar el conjunto de datos para la ubicación deseada
example_loc = train[train['location'] == '-0.51_29.29']

# 3. Calcular la media móvil con una ventana de 2 semanas
rolling_mean = example_loc['SulphurDioxide_SO2_column_number_density_amf'].rolling(window=2).mean()

# 4. Visualizar los resultados
plt.figure(figsize=(15, 7))
rolling_mean.plot()
plt.title('Rolling mean with a window of 2 weeks for SulphurDioxide_SO2_column_number_density_amf', y=1.02, fontsize=15)
plt.xlabel('Week', y=1.05, fontsize=13)
plt.ylabel('SulphurDioxide_SO2_column_number_density_amf', x=1.05, fontsize=13)
plt.show()


In [ ]:
#Excepto por las columnas de índice (latitud, longitud, año y número de semana) y el objetivo (emisión), a todas las columnas les faltan valores.x
# Aplicar codificación one-hot a la columna 'location' y manejar los valores faltantes
# Para el conjunto de entrenamiento (train)
with pd.option_context("display.min_rows", 14):
    # Mostrar la suma de valores faltantes ordenados
    display(train.isna().sum().sort_values())

# Dejar un espacio en blanco entre las visualizaciones
print()

# Para el conjunto de prueba (test)
with pd.option_context("display.min_rows", 14):
    # Mostrar la suma de valores faltantes ordenados
    display(test.isna().sum().sort_values())

In [ ]:
#El resultado de esta operación será una serie de pandas con las combinaciones únicas de latitud
# y longitud como índice y el valor promedio de las emisiones como datos
train.groupby(['latitude', 'longitude']).emission.mean().sort_values()

**añadir el resto**

# Implementacion del modelo

# Librerias Necesarias


In [ ]:
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
# Lista para almacenar los puntajes (RMSE) de cada pliegue
score_list = []

# Crear un objeto LeaveOneGroupOut para la validación cruzada basada en grupos 'year'
kf = LeaveOneGroupOut()

# Iterar a través de los pliegues de validación cruzada
for fold, (idx_tr, idx_va) in enumerate(kf.split(train, groups=train.year)):
    # Dividir los datos en conjuntos de entrenamiento y validación
    X_tr = train.iloc[idx_tr][['longitude', 'latitude', 'week_no']]
    y_tr = train.iloc[idx_tr]['emission']
    X_va = train.iloc[idx_va][['longitude', 'latitude', 'week_no']]
    y_va = train.iloc[idx_va]['emission']

    # Instanciar el modelo (usando RadiusNeighborsRegressor con radio 0)
    model = RadiusNeighborsRegressor(radius=0) # También se podría usar DecisionTreeRegressor u otro modelo aquí

    # Ajustar el modelo con los datos de entrenamiento
    model.fit(X_tr, y_tr)

    # Realizar predicciones en los datos de validación
    y_va_pred = model.predict(X_va)

    # Calcular el RMSE para este pliegue
    rmse = mean_squared_error(y_va, y_va_pred, squared=False)

    # Imprimir el RMSE para este pliegue
    print(f"Fold {fold} year {train.iloc[idx_va].year.iloc[0]}: rmse = {rmse:.2f}")

    # Agregar el RMSE a la lista de puntajes
    score_list.append(rmse)

# Calcular el RMSE promedio de todos los pliegues
rmse = sum(score_list) / len(score_list)

# Imprimir el RMSE promedio
print(f"Overall RMSE: {rmse:.2f}")


In [ ]:
#Creemos un modelo de referencia simple basado en la ubicación y el patrón anual. No utiliza mediciones satelitales ni realiza extrapolaciones.
#La predicción es el promedio de las emisiones de los últimos años para la misma ubicación y semana. No se requiere imputación de valores.
# Crear una instancia del modelo con la configuración deseada (Radius = 0)
model = RadiusNeighborsRegressor(radius=0)  # O DecisionTreeRegressor() si lo prefieres

# Ajustar el modelo a los datos de entrenamiento
model.fit(train[['longitude', 'latitude', 'week_no']], train.emission)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(test[['longitude', 'latitude', 'week_no']])

# Crear una Serie de pandas para las predicciones con un nombre 'emission' y los índices del conjunto de prueba
submission_standard = pd.Series(y_pred, name='emission', index=test.index)

# Guardar las predicciones en un archivo CSV
submission_standard.to_csv('submission_standard.csv')

# Mostrar las predicciones en una Serie de pandas
submission_standard

#Las emisiones tuvieron una caída del 23% en el segundo trimestre de 2020 debido posiblemente a la influencia de el COVID-19, pero se recuperaron con un aumento del 25% en el segundo trimestre de 2021, lo que muestra la influencia significativa de eventos excepcionales en la tendencia y la dificultad de predecir basándose en promedios anteriores.



# Valores faltantes

In [ ]:
# Calcular el porcentaje de valores faltantes por columna en el conjunto de datos (anterior)
missing_percentages = (train.isna().sum() / train.shape[0] * 100).sort_values(ascending=False)[:15]

# Crear un gráfico de barras horizontal
ax = missing_percentages.plot(kind='barh', figsize=(9, 10))

# Configurar el título del gráficow
plt.title('Percentage of Missing Values Per Column in Train Set', fontdict={'size': 15})

# Etiquetar las barras con el porcentaje de valores faltantes
for p in ax.patches:
    percentage = '{:,.0f}%'.format(p.get_width())
    width, height = p.get_width(), p.get_height()
    x = p.get_x() + width + 0.2
    y = p.get_y() + height / 2
    ax.annotate(percentage, (x, y))
plt.show()

#Tendencia del dioxido de sulfuro

In [ ]:
#analizar y visualizar la tendencia temporal de la columna 'SulphurDioxide_SO2_column_number_density_amf'
# Feature engineering para el conjunto de entrenamiento (train)
train_roll_mean = train.groupby('location')[train.columns[5:]].rolling(window=2).mean().reset_index()
train_roll_mean = train_roll_mean.rename(columns={col: col + '_roll_mean' for col in train_roll_mean.columns[2:]})

# Feature engineering para el conjunto de prueba (test)
test['location'] = test[['latitude', 'longitude']].round(2).astype(str).agg('_'.join, axis=1)
test_roll_mean = test.groupby('location')[test.columns[5:]].rolling(window=2).mean().reset_index()
test_roll_mean = test_roll_mean.rename(columns={col: col + '_roll_mean' for col in test_roll_mean.columns[2:]})
test_roll_mean.head()


# Fusión de características ingenierizadas train Y test


In [ ]:
# Fusión de características ingenierizadas con el conjunto de entrenamiento (train)

# Paso 1: Ordenar el conjunto de entrenamiento por 'location', 'year' y 'week_no'.
train = train.sort_values(by=['location', 'year', 'week_no'], ignore_index=True)

# Paso 2: Fusionar el conjunto de entrenamiento ordenado con las características ingenierizadas (train_roll_mean).
train_eng = train.merge(train_roll_mean, how='left', left_index=True, right_index=True)

# Fusión de características ingenierizadas con el conjunto de prueba (test)

# Paso 1: Ordenar el conjunto de prueba por 'location', 'year' y 'week_no'.
test = test.sort_values(by=['location', 'year', 'week_no'], ignore_index=True)

# Paso 2: Fusionar el conjunto de prueba ordenado con las características ingenierizadas (test_roll_mean).
test_eng = test.merge(test_roll_mean, how='left', left_index=True, right_index=True)

# Vista previa del conjunto de prueba con características ingenierizadas
test_eng.head()


In [ ]:
# Feature engineering train
train_roll_mean = train.sort_values(by = ['location', 'year', 'week_no']).groupby(['location'])[train.columns[5:].tolist()].rolling(window = 2).mean().reset_index()
train_roll_mean.drop(['level_1', 'emission', 'location'], axis = 1, inplace = True)
train_roll_mean.columns = [col + '_roll_mean' for col in train_roll_mean.columns]

# Feature engineering test
test.latitude, test.longitude = round(test.latitude, 2), round(test.longitude, 2)
test['location'] = [str(x) + '_' + str(y) for x, y in zip(test.latitude, test.longitude)]
test_roll_mean = test.sort_values(by = ['location', 'year', 'week_no']).groupby(['location'])[test.columns[5:].tolist()].rolling(window = 2).mean().reset_index()
test_roll_mean.drop(['level_1', 'location'], axis = 1, inplace = True)
test_roll_mean.columns =  [col + '_roll_mean' for col in test_roll_mean.columns]
test_roll_mean.head()

In [ ]:
#Train
train_eng = train.sort_values(by = ['location', 'year', 'week_no'], ignore_index = True).merge(train_roll_mean, how = 'left',
                                                                                               left_index=True, right_index=True)
# Test
test_eng = test.sort_values(by = ['location', 'year', 'week_no'], ignore_index = True).merge(test_roll_mean, how = 'left',
                                                                                               left_index=True, right_index=True)
# Preview engineered test set
test_eng.head()

# Seleccion de variables independientes

In [ ]:


X = train_eng.drop(['ID_LAT_LON_YEAR_WEEK', 'location', 'emission'], axis = 1).fillna(0)
y = train_eng.emission

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = SEED)


Se implementó un modelo de regresión utilizando la biblioteca scikit-learn. Se utilizó la validación cruzada Leave-One-Group-Out basada en grupos 'year'. Se optó por el modelo RadiusNeighborsRegressor con radio 0, aunque se proporcionó flexibilidad para usar otros modelos como DecisionTreeRegressor.

El rendimiento del modelo se evaluó utilizando la métrica RMSE (Root Mean Squared Error). Además, se creó un modelo de referencia simple basado en la ubicación y el patrón anual.

In [ ]:

# Instantiating the model
clf = RandomForestRegressor(random_state = SEED, n_jobs=-1)
clf.fit(X_train, y_train)

# Making predictions
y_pred = clf.predict(X_test)

# Measuring the accuracy of the model
print(f'RMSE Score: {mean_squared_error(y_test, y_pred, squared=False)}') #


Es importante señalar que el RMSE es una métrica comúnmente utilizada para evaluar la precisión de modelos de regresión, donde valores más bajos indican una mejor precisión. Este código proporciona una evaluación cuantitativa del rendimiento del modelo RandomForestRegressor en los datos de prueba.

In [ ]:
train_eng.columns

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Supongamos que 'train' es tu DataFrame
temp = train.copy()
temp['quarter'] = temp['week_no'] // 14 + 1
grouped = temp.groupby(['longitude', 'latitude', 'year', 'quarter'])[['emission']].mean()
grouped = grouped.unstack(['longitude', 'latitude'])
delta = 4
one_year_difference = grouped.shift(-delta) - grouped

n_diagrams = 8
_, axs = plt.subplots(2, 4, figsize=(12, 6))
for i in range(n_diagrams):
    ax = axs.ravel()[i]
    ax.scatter(grouped.iloc[i], one_year_difference.iloc[i], c='chocolate', s=3)
    lr = LinearRegression()
    lr.fit(grouped.iloc[i].values.reshape(-1, 1), one_year_difference.iloc[i])
    xs = np.array([[0], [2000]])
    y_pred = lr.predict(xs)
    ax.plot(xs, y_pred, color='orange')
    index = grouped.index[i + delta]
    ax.set_title(f"Q{index[1]}/{index[0]}: {lr.coef_[0]:1.0%}")
    ax.set_xticks([0, 1000, 2000])
plt.suptitle('Year-over-year growth')
plt.tight_layout()
plt.show()


El gráfico de crecimiento año tras año revela patrones significativos en las emisiones promedio a lo largo de diferentes ubicaciones y trimestres. La distribución de puntos muestra una tendencia general, donde la mayoría forma una línea en el gráfico. Cada punto representa una ubicación específica, y las líneas de regresión lineal ajustadas indican la relación entre las emisiones actuales y el cambio en las emisiones después de un año. Las pendientes de estas líneas, expresadas en porcentaje, proporcionan insights sobre las tasas de crecimiento año tras año. Se observa variabilidad significativa entre ubicaciones y trimestres, con algunas áreas experimentando un crecimiento rápido, otras mostrando un crecimiento más moderado, e incluso casos de decrecimiento. La comparación entre trimestres permite identificar patrones estacionales. Este análisis proporciona una comprensión detallada de las dinámicas de emisiones en función de la ubicación y el tiempo, lo que puede ser crucial para la toma de decisiones y la formulación de estrategias ambientales.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Asumiendo que 'train_eng' es tu DataFrame
train_nocovid = train_eng[(train_eng['year'] == 2019) |
                          ((train_eng['year'] == 2020) & (train_eng['week_no'] <= 8)) |
                          ((train_eng['year'] == 2021) & (train_eng['week_no'] > 8))]

# Instanciar un modelo de regresión de árbol de decisión
model = DecisionTreeRegressor()

# Entrenar el modelo con el conjunto de entrenamiento sin datos de COVID-19
model.fit(train_nocovid[['longitude', 'latitude', 'week_no']], train_nocovid['emission'])

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(test[['longitude', 'latitude', 'week_no']])

# Crear una Serie de pandas con las predicciones
submission_nocovid = pd.Series(y_pred, name='emission', index=test.index)

# Mostrar las predicciones junto con 'ID_LAT_LON_YEAR_WEEK' (ordenando las columnas)
result_df = submission_nocovid.to_frame().join(test['ID_LAT_LON_YEAR_WEEK'])
result_df = result_df[['ID_LAT_LON_YEAR_WEEK', 'emission']]

# Mostrar solo 'ID_LAT_LON_YEAR_WEEK' y 'emission' sin el nombre de la columna 'emission'
print(result_df.to_string(index=False, header=False))



Se realizaron análisis comparativos, destacando la influencia de eventos excepcionales, como la pandemia de COVID-19, en las emisiones. Se ajustó un modelo excluyendo datos relacionados con la pandemia y se compararon las predicciones con un modelo estándar.



In [ ]:
import matplotlib.pyplot as plt

# Crear una figura con el tamaño especificado
plt.figure(figsize=(8, 3))

plt.title("Comparing the three test predictions")
plt.xlabel('week of 2022')
plt.ylabel('test prediction')

# Graficar la predicción estándar en azul
plt.plot(range(49), submission_standard.groupby(test['week_no']).mean(), label='standard', lw=2, color='b')

# Graficar la predicción sin datos de COVID-19 en rojo
plt.plot(range(49), submission_nocovid.groupby(test['week_no']).mean(), label='nocovid', lw=2, color='r')

plt.legend()
plt.grid()
plt.show()
